## Case study

Problem description: https://discuss.analyticsvidhya.com/t/hackathon-3-x-predict-customer-worth-for-happy-customer-bank/3802


# Data preprocessing

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('Train_nyOWmfK.csv', encoding = "latin1")
print(data.shape)

In [3]:
del data['ID']
del data['Lead_Creation_Date']

Let's deal with variables one by one.

# DOB

In [4]:
data['DOB'] = pd.to_datetime(data['DOB'])
data['DOB'] = [x.year for x in data['DOB']]

In [5]:
data['DOB'] = np.where(data.DOB <2000, data['DOB'], data['DOB']-100)

In [6]:
data = data[data.DOB>1930]

# Gender

In [9]:
data['Gender'] = data.Gender.replace({'Male':0, 'Female':1})

# City

In [10]:
data['City2'] = data.City.map({'Delhi':'D', 'Bengaluru':'B', "Mumbai":'M', 'Hyderabad':'H', 
                                  'Chennai': 'C', 'Pune': 'P','Kolkata':'K','Ahmedabad':'A',
                              'Jaipur':'J','Gurgaon':'G','Coimbatore':'Co'}).fillna('O')

In [11]:
del data['City']

# Employer_Name

In [13]:
x1 = data.Employer_Name.value_counts()
x1 = x1.to_dict()
x1['0'] = 1 #tam, gdzie nie ma pracodawcy, zakładamy, że to firma 1-osobowa

In [14]:
data['Employees'] = data.Employer_Name.map(x1).fillna(1).astype(int)

In [15]:
data['Employees'] = np.log(data['Employees'])

In [16]:
del data['Employer_Name']

# Monthly_Income

In [17]:
data = data[data.Monthly_Income <= 1200000]

In [18]:
data = data[data.Monthly_Income > 10000]

In [19]:
data['Monthly_Income'] = np.log(data['Monthly_Income'])

# Loan_Amount_Applied Loan_Amount_Submitted

In [22]:
data['Loan_Amount_Applied'] = data.Loan_Amount_Applied.fillna(0)
data['Loan_Amount_Submitted'] = data.Loan_Amount_Submitted.fillna(0)

In [34]:
a = data[data.Loan_Amount_Applied>0]
a = a[a.Loan_Amount_Submitted>0]
a = a[a.Loan_Amount_Submitted <= a.Loan_Amount_Applied]
a = a.assign(iloraz = a.Loan_Amount_Submitted/a.Loan_Amount_Applied)
sr = np.mean(a['iloraz'])
sr

0.935661033874577

In [41]:
ma = np.mean(a['Loan_Amount_Applied'])
ms = np.mean(a['Loan_Amount_Submitted'])
print (ma,ms)

375660.6675193374 329362.0196114258


In [58]:
def imputacja_A(d):
    if (d['Loan_Amount_Applied']==0):
        if (d['Loan_Amount_Submitted'] >= 100):
            return d['Loan_Amount_Submitted']/sr
        else:
            d['Loan_Amount_Submitted'] == 0
    return d['Loan_Amount_Applied']
        
data['Loan_Amount_Applied'] = data.apply(imputacja_A, axis=1)

In [63]:
def imputacja_A1(d):
    if (d['Loan_Amount_Applied']<1000 and d['Loan_Amount_Submitted'] >d['Loan_Amount_Applied']):
        return d['Loan_Amount_Submitted']/sr
    else:
        return d['Loan_Amount_Applied']
        
data['Loan_Amount_Applied'] = data.apply(imputacja_A1, axis=1)

In [60]:
def imputacja_S(d):
    if d['Loan_Amount_Submitted']==0:
        if d['Loan_Amount_Applied'] >=100:
            return d['Loan_Amount_Applied']*sr
        else:
            d['Loan_Amount_Applied'] == 0
    return d['Loan_Amount_Submitted']
        
data['Loan_Amount_Submitted'] = data.apply(imputacja_S, axis=1)

In [64]:
# in case both are 0
data['Loan_Amount_Applied'] = np.where(data.Loan_Amount_Applied==0, ma, data['Loan_Amount_Applied'])
data['Loan_Amount_Submitted'] = np.where(data.Loan_Amount_Submitted==0, ms, data['Loan_Amount_Submitted'])

In [96]:
data['Loan_Amount_Applied'] = np.log(data['Loan_Amount_Applied'])
data['Loan_Amount_Submitted'] = np.log(data['Loan_Amount_Submitted'])

# Var1

In [68]:
data['var11'] = data.Var1.map({'HVYS': 'A','HCXG':'A', 'HAXF':'A', 'HBXB':'A',
                               'HCXD':'B','HCYS':'B','HBXC':'B',
                               'HBXH':'C','HAYT':'C','HBXX':'C',
                               'HAZD':'D','HAVC':'D','HAXA':'D',
                               'HCXF':'E','HBXD':'E',
                               'HAXM':'F',
                               'HBXA':'H',
                               'HAXB':'I','HAXC':'I'})

In [ ]:
del data['Var1']

# Filled_Form

In [73]:
x2 = data.groupby(['Filled_Form'])['Disbursed'].mean()
x2

Filled_Form
N    0.013337
Y    0.022980
Name: Disbursed, dtype: float64

# Device_Type

In [77]:
x1 = data.Device_Type.value_counts()
x1

Web-browser    60650
Mobile         20735
Name: Device_Type, dtype: int64

In [78]:
x2 = data.groupby(['Device_Type'])['Disbursed'].mean()
x2

Device_Type
Mobile         0.012732
Web-browser    0.016636
Name: Disbursed, dtype: float64

# Var2

In [87]:
data['Var2'] = np.where(data['Var2']=='A', 'D', data['Var2'])

# Source

In [93]:
l = ['S122' ,'S133', 'S159','S143','S127','S137','S134']
def Sou(d):
    if d['Source'] in l:
        return d['Source']
    else:
        return 'S0'
        
data['Source'] = data.apply(Sou, axis=1)

# Var4

In [279]:
data['var4'] = data.Var4.astype('str')

# Loan_Tenure_Applied Loan_Tenure_Submitted

In [100]:
data['Loan_Tenure_Applied'] = data.Loan_Tenure_Applied.fillna(0)
data['Loan_Tenure_Submitted'] = data.Loan_Tenure_Submitted.fillna(0)

In [102]:
a = data[data.Loan_Tenure_Applied >0]
ma = np.mean(a.Loan_Tenure_Applied)
a = data[data.Loan_Tenure_Submitted >0]
ms = np.mean(a.Loan_Tenure_Submitted)
print (ma,ms)

3.517545083192189 3.8928126855238716


In [107]:
def imputacja_TA(d):
    if (d['Loan_Tenure_Applied']==0 and d['Loan_Tenure_Submitted'] > 0):
        return d['Loan_Tenure_Submitted']
    else:
        return d['Loan_Tenure_Applied']
        
data['Loan_Tenure_Applied'] = data.apply(imputacja_TA, axis=1)

In [108]:
def imputacja_TS(d):
    if (d['Loan_Tenure_Submitted']==0 and d['Loan_Tenure_Applied'] > 0):
        return d['Loan_Tenure_Applied']
    else:
        return d['Loan_Tenure_Submitted']
        
data['Loan_Tenure_Submitted'] = data.apply(imputacja_TS, axis=1)

In [109]:
# in case both are 0
data['Loan_Tenure_Applied'] = np.where(data.Loan_Tenure_Applied==0, ma, data['Loan_Tenure_Applied'])
data['Loan_Tenure_Submitted'] = np.where(data.Loan_Tenure_Submitted==0, ms, data['Loan_Tenure_Submitted'])

# Salary_Account

In [114]:
l = ['HDFC Bank','ICICI Bank','State Bank of India','Axis Bank',
     'Citibank','Kotak Bank','IDBI Bank','Punjab National Bank',
     'Bank of India ']
def B(d):
    if d['Salary_Account'] in l:
        return d['Salary_Account']
    else:
        return 'other'
        
data['Salary_Account'] = data.apply(B, axis=1)

# Var5

# Existing_EMI

In [126]:
data['Existing_EMI'] = data.Existing_EMI.fillna(0)

In [131]:
data['Existing_EMI'] = np.where(data['Existing_EMI']==0, 0,np.log(data['Existing_EMI']))

/home/kodolamacz/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: RuntimeWarning: divide by zero encountered in log
  if __name__ == '__main__':


# Interest_Rate

In [133]:
data.Interest_Rate.isnull().sum()

53734

In [ ]:
del data['Interest_Rate'] #too many Nans

# Processing_Fee

In [136]:
data.Processing_Fee.isnull().sum()

54040

In [ ]:
del data['Processing_Fee'] #too many Nans

# EMI_Loan_Submitted

In [137]:
data.EMI_Loan_Submitted.isnull().sum()

53734

In [139]:
del data['EMI_Loan_Submitted'] #too many Nans

# LoggedIn

# Mobile_Verified

In [143]:
data = pd.get_dummies(data,drop_first=True)

In [144]:
data.shape

(81385, 54)

In [145]:
data.Disbursed.value_counts()

0    80112
1     1273
Name: Disbursed, dtype: int64

In [146]:
data.isnull().values.any()

False

In [148]:
data.isnull().values.any()

False

In [149]:
data.to_csv('data1.csv')